In [1]:
# 导包
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib
import xgboost as xgb
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

In [2]:
#printing out versions of all packages and libraries and used

print(f'pandas version is: {pd.__version__}')
print(f'numpy version is: {np.__version__}')
print(f'matplotlib version is: {matplotlib.__version__}')
print(f'sklearn version is: {sklearn.__version__}')
print(f'xgboost version is: {xgb.__version__}')

pandas version is: 1.1.5
numpy version is: 1.19.5
matplotlib version is: 2.0.2
sklearn version is: 0.24.2
xgboost version is: 1.5.2


In [16]:
# data = pd.read_csv(r'content/numpy-/美国油田/train_data_补.csv')
# data.head()
# np.count_nonzero(data.Formation.unique()) # 14 中Formation
# data.Formation.unique()
# len(data['Well Name'].unique())
# data.columns


Index(['Formation', 'Well Name', 'Depth', 'GR', 'ILD_log10', 'DeltaPHI',
       'PHIND', 'PE', 'NM_M', 'RELPOS', 'Facies'],
      dtype='object')

In [2]:
# data1 = pd.read_csv(r'D:\workspace\pycharm\welllogAI\data\处理后的FORCE数据集\force-train.csv')
data1 = pd.read_csv("../data/ciflog_scalared_fetures_med9.csv")

In [6]:
fea = data1.columns.drop(['Well Name', 'Formation'])
fea

Index(['Depth', 'GR', 'INPEFA', 'AC', 'SP', 'GR_MED_3', 'GR_MED_5', 'GR_MED_7',
       'GR_MED_9', 'GR_MED_11', 'GR_MED_13', 'GR_MED_15', 'GR_MED_17',
       'GR_MED_19'],
      dtype='object')

In [4]:
print('----------------------处理后的 FORCE 数据集---------------------------')
print(data1.sample(5))
print(f'特征名称：{data1.columns}')
print(f'行列:{data1.shape}')
print(f'共有{len(data1["Formation"].unique())}种地层,所有地层名称：{data1["Formation"].unique()}')
print(f'共有{len(data1["WELL_encoded"].unique())}口井,所有井名称：{data1["WELL_encoded"].unique()}')
print(f'共有{len(data1["LITHOLOGY"].unique())}种岩性,所有岩性名称：{data1["LITHOLOGY"].unique()}')
print(f'共有{len(data1["GROUP_encoded"].unique())}种group,所有group名称：{data1["GROUP_encoded"].unique()}')
data1.LITHOLOGY.unique()

----------------------处理后的 FORCE 数据集---------------------------
          Depth        GR    INPEFA        AC        SP  Well Name  Formation  \
33302 -0.648533 -0.196459  1.947425  0.004728 -0.395807          6          2   
28818  0.124633 -0.806328 -0.960790  0.005003 -0.252663          5          0   
34541 -0.298532  0.829419 -0.199405  0.004714  0.128416          6          3   
3931   0.980568 -0.340667  0.429508  0.004688 -0.402720          0          5   
6717   0.215877  0.110918  1.078388  0.004681 -0.192663          1          2   

       GR_MED_3  GR_MED_5  GR_MED_7  GR_MED_9  GR_MED_11  GR_MED_13  \
33302 -0.196459 -0.192380 -0.192380 -0.196459  -0.196459  -0.196459   
28818 -0.806328 -0.806328 -0.792173 -0.785805  -0.759542  -0.755392   
34541  0.829419  0.848144  0.848144  0.848144   0.829419   0.780366   
3931  -0.386067 -0.402593 -0.412892 -0.421130  -0.423249  -0.423249   
6717   0.110918  0.120674  0.120674  0.120674   0.120674   0.120674   

       GR_MED_15  GR_M

KeyError: 'WELL_encoded'

In [4]:
def drop_columns(data, *args):

    '''
    function used to drop columns.
    args::
      data:  dataframe to be operated on
      *args: a list of columns to be dropped from the dataframe

    return: returns a dataframe with the columns dropped
    '''

    columns = []
    for _ in args:
        columns.append(_)

    data = data.drop(columns, axis=1)

    return data

def process(data):

    '''
    function to process dataframe by replacing missing, infinity values with -999

    args::
      data:  dataframe to be operated on

    returns dataframe with replaced values
    '''

    cols = list(data.columns)
    for _ in cols:

        data[_] = np.where(data[_] == np.inf, -999, data[_])
        data[_] = np.where(data[_] == np.nan, -999, data[_])
        data[_] = np.where(data[_] == -np.inf, -999, data[_])

    return data

def show_evaluation(pred, true):

  '''

  function to show model performance and evaluation
  args:
    pred: predicted value(a list)
    true: actual values (a list)

  prints the custom metric performance, accuracy and F1 score of predictions

  '''

  print(f'Default score: {score(true.values, pred)}')
  print(f'Accuracy is: {accuracy_score(true, pred)}')
  print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')


#Paulo Bestagini's feature augmentation technique from SEG 2016 ML competition
#Link : https://github.com/seg/2016-ml-contest/tree/master/ispl

# Feature windows concatenation function
def augment_features_window(X, N_neig):

    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug

# Feature gradient computation function
def augment_features_gradient(X, depth):

    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff

    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))

    return X_grad

# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):

    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)

    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])

    return X_aug, padded_rows

def score(y_true, y_pred):

    '''
    custom metric used for evaluation
    args:
      y_true: actual prediction
      y_pred: predictions made
    '''

    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [5]:
PWD = 'Force-2020-Machine-Learning-competition/lithology_competition/data'
data_dir = 'data/FORCE_2020_TRAIN.csv'

In [ ]:
A = np.load(PWD + 'penalty_matrix.npy')
data = pd.read_csv(data_dir)

In [ ]:



class Model():

    def __init__(self, train, test) -> None:
        self.train = train
        self.test = test

    def __call__(self, plot = True):

      return self.fit(plot)



    def preprocess(self, train, test):

        '''
        method to prepare datasets for training and predictions
        accepts both the train and test dataframes as arguments

        returns the prepared train, test datasets along with the
        lithology labels and numbers which is needed for preparing
        the submission file

        '''

        #concatenating both train and test datasets for easier and uniform processing

        ntrain = train.shape[0]
        ntest = test.shape[0]
        target = train.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
        df = pd.concat((train, test)).reset_index(drop=True)

        #mapping the lithology labels to ordinal values for better modelling

        lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']

        lithology_numbers = {30000: 0,
                        65030: 1,
                        65000: 2,
                        80000: 3,
                        74000: 4,
                        70000: 5,
                        70032: 6,
                        88000: 7,
                        86000: 8,
                        99000: 9,
                        90000: 10,
                        93000: 11}

        lithology1 = lithology.map(lithology_numbers)

        #implementing Bestagini's augmentation procedure

        train_well = train.WELL.values
        train_depth = train.DEPTH_MD.values

        test_well = test.WELL.values
        test_depth = test.DEPTH_MD.values
        '''to be continued...
        #this was done here for ease as the datasets would undergo some transformations
        #that would make it uneasy to perform the augmentation technique'''



        print(f'shape of concatenated dataframe before dropping columns {df.shape}')

        cols = ['FORCE_2020_LITHOFACIES_CONFIDENCE', 'SGR', 'DTS', 'RXO', 'ROPA'] #columns to be dropped
        df = drop_columns(df, *cols)
        print(f'shape of dataframe after dropping columns {df.shape}')
        print(f'{cols} were dropped')

        #Label encoding the GROUP, FORMATION and WELLS features as these improved the performance of the models on validations

        df['GROUP_encoded'] = df['GROUP'].astype('category')
        df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes
        df['FORMATION_encoded'] = df['FORMATION'].astype('category')
        df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes
        df['WELL_encoded'] = df['WELL'].astype('category')
        df['WELL_encoded'] = df['WELL_encoded'].cat.codes
        print(f'shape of dataframe after label encoding columns {df.shape}')


        #FURTHER PREPRATION TO SPLIT DATAFRAME INTO TRAIN AND TEST DATASETS AFTER PREPRATION
        print(f'Splitting concatenated dataframe into training and test datasets...')
        df = df.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)
        print(df.shape)

        df = df.fillna(-999)
        df = process(df)
        data = df.copy()

        train2 = data[:ntrain].copy()
        train2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)

        test2 = data[ntrain:(ntest+ntrain)].copy()
        test2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
        test2 = test2.reset_index(drop=True)

        traindata = train2
        testdata = test2

        print(f'Shape of train and test datasets before augmentation {traindata.shape, testdata.shape}')

        traindata1, padded_rows = augment_features(pd.DataFrame(traindata).values, train_well, train_depth)
        testdata1, padded_rows = augment_features(pd.DataFrame(testdata).values, test_well, test_depth)

        print(f'Shape of train and test datasets after augmentation {traindata1.shape, testdata1.shape}')

        return traindata1, testdata1, lithology1, lithology_numbers
    def fit(self, plot):

      '''
      method to train model and make predictions

      returns the test predictions, trained model, and lithology numbers used for making the submission file
      '''

      traindata1, testdata1, lithology1, lithology_numbers = self.preprocess(self.train, self.test)

      #using a 10-fold stratified cross-validation technique and seting the shuffle parameter to true
      #as this improved the validation performance better

      split = 10
      kf = StratifiedKFold(n_splits=split, shuffle=True)

      open_test = np.zeros((len(testdata1), 12))

      models = []

      model_1dcnn = self.get_model_1dcnn()
      model_lstm = self.get_model_lstm()
      model_bilstm = self.get_model_bilstm()
      model_tcn = self.get_model_tcn()
      models.append(model_1dcnn, model_lstm, model_bilstm, model_tcn)

      # TODO: 添加 4 个 模型
      model = models[0]
      # model.summary()
      model.fit(traindata1, lithology1)


      i = 1
      for (train_index, test_index) in kf.split(pd.DataFrame(traindata1), pd.DataFrame(lithology1)):
        X_train, X_test = pd.DataFrame(traindata1).iloc[train_index], pd.DataFrame(traindata1).iloc[test_index]
        Y_train, Y_test = pd.DataFrame(lithology1).iloc[train_index],pd.DataFrame(lithology1).iloc[test_index]

        model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=100)
        prediction = model.predict(X_test)
        print(show_evaluation(prediction, Y_test))

        print(f'-----------------------FOLD {i}---------------------')
        i+=1

        open_test += model.predict_proba(pd.DataFrame(testdata1))

      open_test= pd.DataFrame(open_test/split)

      open_test = np.array(pd.DataFrame(open_test).idxmax(axis=1))

      print('---------------CROSS VALIDATION COMPLETE')
      print('----------------TEST EVALUATION------------------')


      if plot: self.plot_feat_imp(model)
      return open_test, model, lithology_numbers


    def plot_feat_imp(self, model):
        feat_imp = pd.Series(model.feature_importances_).sort_values(ascending=False)
        plt.figure(figsize=(12,8))
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

    def make_submission_file(self, filename):

      '''
      method to train model, make prediction and create submission file
      args::
        filename: name to save submission file as (string)
      '''

      self.filename = filename

      prediction, model, lithology_numbers = self.fit(plot=False)

      path = PWD

      test = pd.read_csv(PWD + 'Test.csv', sep=';')

      category_to_lithology = {y:x for x,y in lithology_numbers.items()}
      test_prediction_for_submission = np.vectorize(category_to_lithology.get)(prediction)
      np.savetxt(path+filename+'.csv', test_prediction_for_submission, header='lithology', fmt='%i')

    def get_model_lstm(self):
        pass

    def get_model_bilstm(self):
        pass

    def get_model_tcn(self):
        pass

    def get_model_1dcnn(self):
        pass